# 🧠 Deutsch Algorithm — Qiskit Implementation
**Author:** Satya Pal  
**Repository:** [quantum-satya/quantum-algorithms](https://github.com/quantum-satya/quantum-algorithms)

---

## 🎯 Objective
The **Deutsch Algorithm** is one of the earliest examples showing how quantum computing  
can outperform classical computing — solving a specific problem with **one oracle call**  
instead of two.

We are given a black-box Boolean function `f(x): {0,1} → {0,1}`  
and we must determine whether `f` is:
- **Constant:** outputs the same value for all x  
- **Balanced:** outputs 0 for one input and 1 for the other  

The quantum version solves this in **a single query**!


In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


## ⚙️ Step 1: Define the Oracle
The oracle represents the secret function `f(x)`.

We’ll implement two examples:
- **Constant oracle**: returns 0 for all x  
- **Balanced oracle**: returns x (so f(0)=0, f(1)=1)


In [ ]:
def constant_oracle():
    qc = QuantumCircuit(2)
    # does nothing -> always returns 0
    return qc

def balanced_oracle():
    qc = QuantumCircuit(2)
    qc.cx(0, 1)  # flip output when input=1
    return qc


## ⚡ Step 2: Build Deutsch Circuit
Algorithm steps:
1. Initialize qubits to |0⟩|1⟩
2. Apply Hadamard gates to create superposition
3. Apply the oracle
4. Apply another Hadamard to the first qubit
5. Measure the first qubit — it tells us:
   - 0 → constant
   - 1 → balanced


In [ ]:
def deutsch_algorithm(oracle):
    qc = QuantumCircuit(2, 1)

    # Step 1: Initialize |0>|1>
    qc.x(1)
    qc.barrier()

    # Step 2: Hadamards
    qc.h([0, 1])
    qc.barrier()

    # Step 3: Apply oracle
    qc.compose(oracle, inplace=True)
    qc.barrier()

    # Step 4: Hadamard on first qubit
    qc.h(0)

    # Step 5: Measure
    qc.measure(0, 0)
    return qc


In [ ]:
simulator = Aer.get_backend('qasm_simulator')

# Constant case
qc_const = deutsch_algorithm(constant_oracle())
result_const = execute(qc_const, simulator, shots=1024).result()
counts_const = result_const.get_counts()

# Balanced case
qc_bal = deutsch_algorithm(balanced_oracle())
result_bal = execute(qc_bal, simulator, shots=1024).result()
counts_bal = result_bal.get_counts()

# Visualize
fig, ax = plt.subplots(1, 2, figsize=(10,4))
plot_histogram(counts_const, ax=ax[0])
plot_histogram(counts_bal, ax=ax[1])
ax[0].set_title("Constant Oracle")
ax[1].set_title("Balanced Oracle")
plt.show()


## 🧩 Step 3: Interpretation
- If the first qubit is measured as **0 → f(x) is constant**
- If measured as **1 → f(x) is balanced**

✅ **Quantum advantage:**  
The Deutsch algorithm determines the type of function with **a single query**,  
while classical algorithms need **two**.
